# Basic Data Cleaning
link: https://machinelearningmastery.com/basic-data-cleaning-for-machine-learning/

This tutorial is divided into seven parts; they are:
1. Messy Datasets
2. Identify Columns That Contain a Single Value
3. Delete Columns That Contain a Single Value
4. Consider Columns That Have Very Few Values
5. Remove Columns That Have A Low Variance
6. Identify Rows that Contain Duplicate Data
7. Delete Rows that Contain Duplicate Data


Data cleaning refers to identifying and correcting errors in the dataset that may negatively impact a predictive model. There are many types of errors that exist in a dataset, although some of the simplest errors
include columns that don’t contain much information and duplicated rows. Before we dive into identifying and correcting messy data, let’s define some messy datasets

## Identify Columns That Contain a Single Value

Columns that have a single observation or value are probably useless for modeling. These columns or predictors are referred to zero-variance predictors as if we measured the variance
(average value from the mean), it would be zero.

Columns that have a single value for all rows do not contain any information for modeling.
Depending on the choice of data preparation and modeling algorithms, variables with a single value can also cause errors or unexpected results. You can detect rows that have this property using the `unique()` NumPy function that will report the number of unique values in each
column

In [1]:
# summarize the percentage of unique values for each column using numpy
from numpy import loadtxt
from numpy import unique
# load the dataset
data = loadtxt( '../../Datasets/oil-spill.csv' , delimiter= ',' )
# summarize the number of unique values in each column
for i in range(data.shape[1]):
  num = len(unique(data[:, i]))
  percentage = float(num) / data.shape[0] * 100
  print( ' %d, %d, %.1f%% ' % (i, num, percentage))

 0, 238, 25.4% 
 1, 297, 31.7% 
 2, 927, 98.9% 
 3, 933, 99.6% 
 4, 179, 19.1% 
 5, 375, 40.0% 
 6, 820, 87.5% 
 7, 618, 66.0% 
 8, 561, 59.9% 
 9, 57, 6.1% 
 10, 577, 61.6% 
 11, 59, 6.3% 
 12, 73, 7.8% 
 13, 107, 11.4% 
 14, 53, 5.7% 
 15, 91, 9.7% 
 16, 893, 95.3% 
 17, 810, 86.4% 
 18, 170, 18.1% 
 19, 53, 5.7% 
 20, 68, 7.3% 
 21, 9, 1.0% 
 22, 1, 0.1% 
 23, 92, 9.8% 
 24, 9, 1.0% 
 25, 8, 0.9% 
 26, 9, 1.0% 
 27, 308, 32.9% 
 28, 447, 47.7% 
 29, 392, 41.8% 
 30, 107, 11.4% 
 31, 42, 4.5% 
 32, 4, 0.4% 
 33, 45, 4.8% 
 34, 141, 15.0% 
 35, 110, 11.7% 
 36, 3, 0.3% 
 37, 758, 80.9% 
 38, 9, 1.0% 
 39, 9, 1.0% 
 40, 388, 41.4% 
 41, 220, 23.5% 
 42, 644, 68.7% 
 43, 649, 69.3% 
 44, 499, 53.3% 
 45, 2, 0.2% 
 46, 937, 100.0% 
 47, 169, 18.0% 
 48, 286, 30.5% 
 49, 2, 0.2% 


We can update the example to only summarize those variables that have unique values that
are less than 1 percent of the number of rows.

In [2]:
# summarize the number of unique values in each column
for i in range(data.shape[1]):
    num = len(unique(data[:, i]))
    percentage = float(num) / data.shape[0] * 100
    if percentage < 1:
        print( ' %d, %d, %.1f%% ' % (i, num, percentage))

 21, 9, 1.0% 
 22, 1, 0.1% 
 24, 9, 1.0% 
 25, 8, 0.9% 
 26, 9, 1.0% 
 32, 4, 0.4% 
 36, 3, 0.3% 
 38, 9, 1.0% 
 39, 9, 1.0% 
 45, 2, 0.2% 
 49, 2, 0.2% 


In [3]:
# delete columns where number of unique values is less than 1% of the rows
from pandas import read_csv
# load the dataset
df = read_csv( '../../Datasets/oil-spill.csv' , header=None)
print(df.shape)
# get number of unique values for each column
counts = df.nunique()
# record columns to delete
to_del = [i for i,v in enumerate(counts) if (float(v)/df.shape[0]*100) < 1]
print(to_del)
# drop useless columns
df.drop(to_del, axis=1, inplace=True)
print(df.shape)

(937, 50)
[21, 22, 24, 25, 26, 32, 36, 38, 39, 45, 49]
(937, 39)


## Remove Columns That Have A Low Variance
Another approach to the problem of removing columns with few unique values is to consider
the variance of the column. Recall that the variance is a statistic calculated on a variable as the
average squared difference of values in the sample from the mean. The variance can be used as a
filter for identifying columns to be removed from the dataset. A column that has a single value
has a variance of 0.0, and a column that has very few unique values may have a small variance.

The `VarianceThreshold` class from the scikit-learn library supports this as a type of feature
selection. An instance of the class can be created and we can specify the threshold argument,
which defaults to 0.0 to remove columns with a single value. It can then be fit and applied
to a dataset by calling the fit transform() function to create a transformed version of the
dataset where the columns that have a variance lower than the threshold have been removed
automatically.

In [4]:
from sklearn.feature_selection import VarianceThreshold
# split data into inputs and outputs
data = df.values
X = data[:, :-1]
y = data[:, -1]
print(X.shape, y.shape)
# define the transform
transform = VarianceThreshold()
# transform the input data
X_sel = transform.fit_transform(X)
print(X_sel.shape)

(937, 38) (937,)
(937, 38)


In [ ]:
from numpy import arange
from matplotlib import pyplot
# define thresholds to check
thresholds = arange(0.0, 0.55, 0.05)
# apply transform with each threshold
results = list()
for t in thresholds:
    # define the transform
    transform = VarianceThreshold(threshold=t)
    # transform the input data
    X_sel = transform.fit_transform(X)
    # determine the number of input features
    n_features = X_sel.shape[1]
    print( ' >Threshold=%.2f, Features=%d ' % (t, n_features))
    # store the result
    results.append(n_features)
# plot the threshold vs the number of selected features
pyplot.plot(thresholds, results)
pyplot.show()

## Identify Rows That Contain Duplicate Data

In [8]:
# locate rows of duplicate data
from pandas import read_csv
# load the dataset
df = read_csv( '../../Datasets/iris.csv' , header=None)
# calculate duplicates
dups = df.duplicated()
# report if there are any duplicates
print(dups.any())
# list all duplicate rows
print(df[dups])

True
       0    1    2    3               4
34   4.9  3.1  1.5  0.1     Iris-setosa
37   4.9  3.1  1.5  0.1     Iris-setosa
142  5.8  2.7  5.1  1.9  Iris-virginica


In [9]:
print(df.shape)
# delete duplicate rows
df.drop_duplicates(inplace=True)
print(df.shape)

(150, 5)
(147, 5)
